In [ ]:
from test_configuration import firestore_client, plaid_client
from plaid.model.institutions_get_by_id_request import InstitutionsGetByIdRequest
from plaid.model.institutions_get_by_id_response import InstitutionsGetByIdResponse
from plaid.model.institutions_get_by_id_request_options import InstitutionsGetByIdRequestOptions
from plaid.model.institutions_get_request import InstitutionsGetRequest
from plaid.model.institutions_get_request_options import InstitutionsGetRequestOptions
from plaid.model.institutions_get_response import InstitutionsGetResponse
from plaid.model.country_code import CountryCode
from plaid import ApiException
import json
from typing import Union, Dict

In [ ]:
def institutions_get_by_id(institution_id: str):
    try:
        options = InstitutionsGetByIdRequestOptions(
            include_optional_metadata=True,
        )
        request = InstitutionsGetByIdRequest(
            institution_id=institution_id,
            country_codes=[CountryCode('US')],
            options=options,
        )
        response: InstitutionsGetByIdResponse = plaid_client.institutions_get_by_id(request)
        response_dict = response.to_dict()

        return response_dict
    except ApiException as e:
        exceptions: dict = json.loads(e.body)

        return exceptions

In [ ]:
def institutions_get(count:int, offset: int):
    try:
        options = InstitutionsGetRequestOptions(
            include_optional_metadata=True,
        )
        request = InstitutionsGetRequest(
            country_codes=[CountryCode('US')],
            count=count,
            offset=offset,
            options=options,

        )
        response: InstitutionsGetResponse = plaid_client.institutions_get(request)
        response_dict = response.to_dict()

        return response_dict
    except ApiException as e:
        exceptions: dict = json.loads(e.body)

        return exceptions

In [ ]:
def update_institution(request: Dict):
    # TODO: ^change the type of request to `flask.Request`

    try:
        # TODO: uncomment for the release
        # data = request.data
        # data_dict: dict = json.loads(data)
        # institution_id = data_dict.get('institution_id')
        institution_id = request.get('institution_id')
        institutions_get_result = institutions_get_by_id(institution_id)
        institution: Union[Dict, None] = institutions_get_result.get('institution')

        if institution is None:
            error_code = institutions_get_result.get('error_code')
            error_message = institutions_get_result.get('error_message')

            # TODO: uncomment below for release
            # return make_response(jsonify(status=error_code, error_message=error_message), 404)
            return {'status': error_code, 'error_message': error_message}

        # if fetched institution data exists, either create or update firestore data
        institution_collections = firestore_client.collection('institutions')
        institution_doc = institution_collections.document(institution_id)
        institution_snapshot = institution_doc.get()

        if institution_snapshot.exists:
            institution_doc.update(institution)
        else:
            institution_doc.create(institution)
        
        # TODO: uncomment below for release
        # return make_response(jsonify(status=200,message='Successfully updated institution data'))

        return 'Created new institution document'
    except Exception as e:
        print(str(e))

        # TODO: uncomment below for release
        # return make_response(jsonify(status=404, error_message=str(e)), 404)

In [ ]:
# data = {'institution_id': 'ins_3'}
# result = update_institution(data)
# result

institutions_get_by_id('ins_3')
# institutions_get(10,0)